# CRM Agent with Custom Tools

This notebook demonstrates how to create an AI agent with custom CRM tools that can:
- Create new customer records
- Retrieve customer records by ID
- Get all customer records
- Search customers by name, email, or city

In [2]:
import asyncio
import os
import sys
from pathlib import Path

# Add workspace root to Python path for importing crm_tools
workspace_root = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
sys.path.insert(0, str(workspace_root))

from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity import AzureCliCredential

# Import CRM tools
from crm_tools.tools import (
    create_customer,
    get_customer_by_id,
    get_all_customers,
    search_customers,
    customers_db
)

## View Initial Customer Data

The database is pre-populated with 10 Australian customers.

In [3]:
print(f"Total customers in database: {len(customers_db)}\n")
for customer in customers_db:
    print(f"ID={customer['id']}, Name={customer['name']}, City={customer['city']}, "
          f"Credit=${customer['credit_limit']}, Status={customer['active_status']}, Email={customer['email']}")

Total customers in database: 10

ID=1, Name=Liam O'Connor, City=Sydney, Credit=$1129, Status=active, Email=liam.oconnor@example.com.au
ID=2, Name=Charlotte Mitchell, City=Melbourne, Credit=$2306, Status=active, Email=charlotte.mitchell@example.com.au
ID=3, Name=Jack Thompson, City=Brisbane, Credit=$6637, Status=active, Email=jack.thompson@example.com.au
ID=4, Name=Olivia Williams, City=Perth, Credit=$9496, Status=inactive, Email=olivia.williams@example.com.au
ID=5, Name=Noah Fraser, City=Adelaide, Credit=$4756, Status=active, Email=noah.fraser@example.com.au
ID=6, Name=Amelia Robertson, City=Gold Coast, Credit=$7758, Status=active, Email=amelia.robertson@example.com.au
ID=7, Name=William Chen, City=Canberra, Credit=$6612, Status=inactive, Email=william.chen@example.com.au
ID=8, Name=Isla Murphy, City=Hobart, Credit=$3214, Status=active, Email=isla.murphy@example.com.au
ID=9, Name=James Kelly, City=Darwin, Credit=$2669, Status=active, Email=james.kelly@example.com.au
ID=10, Name=Mia And

## Setup CRM Agent

Create an agent with all CRM tools attached.

In [4]:
credential = AzureCliCredential()
client = AzureOpenAIResponsesClient(
    project_endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    deployment_name=os.environ["AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME"],
    credential=credential,
)

agent = client.as_agent(
    name="CRMAgent",
    instructions="You are a helpful CRM assistant. Use the available tools to help users manage customer records. Always provide clear and formatted responses.",
    tools=[create_customer, get_customer_by_id, get_all_customers, search_customers],
)

## Demo 1: Show All Customers

In [5]:
result = await agent.run("Show me all customers")
print(f"Agent: {result}")

Agent: Here are all the customer records:

1. **Liam O'Connor**
   - City: Sydney
   - Credit Limit: $1,129
   - Status: Active
   - Email: liam.oconnor@example.com.au

2. **Charlotte Mitchell**
   - City: Melbourne
   - Credit Limit: $2,306
   - Status: Active
   - Email: charlotte.mitchell@example.com.au

3. **Jack Thompson**
   - City: Brisbane
   - Credit Limit: $6,637
   - Status: Active
   - Email: jack.thompson@example.com.au

4. **Olivia Williams**
   - City: Perth
   - Credit Limit: $9,496
   - Status: Inactive
   - Email: olivia.williams@example.com.au

5. **Noah Fraser**
   - City: Adelaide
   - Credit Limit: $4,756
   - Status: Active
   - Email: noah.fraser@example.com.au

6. **Amelia Robertson**
   - City: Gold Coast
   - Credit Limit: $7,758
   - Status: Active
   - Email: amelia.robertson@example.com.au

7. **William Chen**
   - City: Canberra
   - Credit Limit: $6,612
   - Status: Inactive
   - Email: william.chen@example.com.au

8. **Isla Murphy**
   - City: Hobart
  

## Demo 2: Find Customers in Sydney

In [6]:
result = await agent.run("Find customers in Sydney")
print(f"Agent: {result}")

Agent: Here is the customer record located in Sydney:

- **Name:** Liam O'Connor
- **City:** Sydney
- **Credit Limit:** $1129
- **Status:** Active
- **Email:** liam.oconnor@example.com.au

Let me know if you need further assistance!


## Demo 3: Get Customer by ID

In [7]:
result = await agent.run("Get customer with ID 5")
print(f"Agent: {result}")

Agent: Here are the details for the customer with ID 5:

- **Name:** Noah Fraser
- **City:** Adelaide
- **Credit Limit:** $4756
- **Status:** Active
- **Email:** [noah.fraser@example.com.au](mailto:noah.fraser@example.com.au)


## Demo 4: Search by Email

In [8]:
result = await agent.run("Search for customers with email containing 'williams'")
print(f"Agent: {result}")

Agent: Here is the customer whose email contains "williams":

- **Name:** Olivia Williams
- **City:** Perth
- **Credit Limit:** $9496
- **Status:** Inactive
- **Email:** olivia.williams@example.com.au

If you need further assistance, feel free to ask!


## Demo 5: Create a New Customer

In [9]:
result = await agent.run("Create a new customer named Emma Davis from Cairns with email emma.davis@example.com.au, credit limit of 5000, and active status")
print(f"Agent: {result}")

Agent: Emma Davis has been successfully added as a new customer. Here are the details:

- **Customer ID**: 11
- **Name**: Emma Davis
- **City**: Cairns
- **Credit Limit**: $5000.00
- **Status**: Active
- **Email**: emma.davis@example.com.au

If you need any further assistance, feel free to ask!


## Demo 6: Search by Name (Partial Match)

In [10]:
result = await agent.run("Find all customers with 'jack' in their name")
print(f"Agent: {result}")

Agent: Here is the customer record with "jack" in their name:

- **Name:** Jack Thompson
- **City:** Brisbane
- **Credit Limit:** $6637
- **Status:** Active
- **Email:** jack.thompson@example.com.au

Is there anything else you would like to do?


## Demo 7: Complex Query - Multiple Criteria

In [11]:
result = await agent.run("Show me all customers from Melbourne and tell me their total count")
print(f"Agent: {result}")

Agent: Here is the customer from Melbourne:

- **Name:** Charlotte Mitchell
- **City:** Melbourne
- **Credit Limit:** $2,306
- **Status:** Active
- **Email:** charlotte.mitchell@example.com.au

**Total Count of Customers from Melbourne:** 1


## Verify New Customer Was Added

In [12]:
print(f"Total customers now: {len(customers_db)}")
result = await agent.run("Show me the customer with ID 11")
print(f"Agent: {result}")

Total customers now: 11
Agent: Here is the customer information for ID 11:

- **Name**: Emma Davis
- **City**: Cairns
- **Credit Limit**: $5000.00
- **Status**: Active
- **Email**: emma.davis@example.com.au
